In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy.stats import chi2_contingency
from datetime import datetime as datet

'fields.csv' |
'works.csv' |
 'values.csv'

# Baseline

In [2]:
df_fi = pd.read_csv('fields_without_humus.csv', encoding='utf-8', sep=',', decimal='.')
#df_fi = df_fi.drop(columns=['Unnamed: 18'])
df_fi = df_fi.drop_duplicates(keep='first')

df_fi.head()

FileNotFoundError: [Errno 2] No such file or directory: 'fields_without_humus.csv'

In [ ]:
df_fi.dtypes

Добавим тип почвы

In [ ]:
dict_reg = {
    "Алтай": 'Черноземы', #Черноземы выщелоченные
    "Курск": 'Черноземы', #Черноземы типичные
    "Калуга": 'Дерново-подзолистые',
    "Тюмень": 'Дерново-подзолистые',
    "Рязань": 'Серые лесные',
    "СПб": 'Торфяно- и торфянисто-подзолисто-глеевые',
    "Башкирия": 'Серые лесные',
    "Татарстан": 'Серые лесные',
    "Оренбург": 'Черноземы', #Черноземы южные
    "Самара": 'Черноземы', #Черноземы обыкновенные
    "Новосибирск": 'Черноземы', #Черноземы выщелоченные
    "Москва": 'Дерново-подзолистые',
    "Воронеж": 'Черноземы' #Черноземы выщелоченные/типичные
}
df_fi['soil'] = df_fi['region'].map(dict_reg)

In [ ]:
df_fi['season'].value_counts()

### Данных по обоим годам достаточно для анализа

In [ ]:
df_fi[df_fi['region'] == 'Башкирия']

### Все микроэлементы по нулям => мы ничего не можем с этим сделать (это 100% ошибка), значит, дропаем, по Башкирии данных немного

In [ ]:
df_fi = df_fi[df_fi['region'] != 'Башкирия']

_______

### Обработаем гумус

In [ ]:
df_fi['humus'].describe()

### Гумус в процентном соотношении, значит больше 100% не может быть + удалим выбросы трехквантильным размахом

In [ ]:
def fkkk():
    Q1 = df_fi['humus'].quantile(0.25)
    Q3 = df_fi['humus'].quantile(0.75)
    IQR = Q3 - Q1

    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    df_fi['humus'] = df_fi['humus'].apply(lambda x: x if (x >= lower_bound) & (x <= upper_bound) else np.nan)
    print(df_fi['humus'].isna().sum())
    df_fi['humus'].describe()


fkkk()
fkkk()

In [ ]:
df_fi['humus'].describe()

In [ ]:
f, ax = plt.subplots(1, 2, figsize=(12, 6))
sns.boxplot(df_fi['humus'], ax=ax[0])
sns.histplot(df_fi['humus'], ax=ax[1])
plt.yscale('log')

______

In [ ]:
dfw = pd.read_csv('/works.csv', sep=';', encoding='1251')
dfw = dfw.drop_duplicates(keep='first')

df_fi['key'] = df_fi.apply(lambda x: (x.season, x.field), axis=1)
dfw['key'] = dfw.apply(lambda x: (x.season, x.field), axis=1)

In [ ]:
print(len(dfw.key.unique()), len(df_fi.key.unique()))

### Имеем разное количество ключей между таблицами поля и выполненных над ними работ, значит по каким-то полям не имеем информации о проводимых над ними работах



______

In [ ]:
order = ['Калуга', 'Воронеж', "Рязань", "Новосибирск", "Курск", "Татарстан", "Оренбург", "Москва", "Алтай", "Самара",
       "Тюмень", "СПб"]
dt = df_fi['region'].value_counts()
dtt = []
for reg in dt.keys():
    dtt.extend([reg] * dt[reg])
df_v = pd.DataFrame({'Регион': dtt})

f, ax = plt.subplots(2, 1, figsize=(13, 6))
# sns.histplot(df_v['Регион'])

sns.countplot(data=df_fi, x='region', hue='season', legend='auto', ax=ax[0], order=order)
ax[0].set_title('распределение записей по регионам(с учетом сезона) в абсолютном значении')

sns.countplot(data=df_fi, x='region', hue='season', legend='auto', ax=ax[1], stat='percent', order=order)
ax[1].set_title('Распределение записей по регионам(с учетом сезона) в процентах')

plt.subplots_adjust(hspace=0.5)
plt.show()
print(dt)

____________

Закончили с удалением данных

### Заполним наши NaN

In [ ]:
# Группируем данные и рассчитываем среднее по 'humus'
hum_mean = df_fi.groupby(['region'])['humus'].mean()
# Заполняем пропуски в 'humus'
df_f = df_fi.copy() # для поиска корреляций без лишних медианных значений
df_fi['humus'] = df_fi['humus'].fillna(
    df_fi.apply(lambda x: hum_mean.get((x['region'])), axis=1)
)
# проверим на наличие nan после заполнения, вдруг ошибка какая была
print(df_fi.isna().sum())

### Нельзя дропать crop_yearold, так как в нем указаны значения только для многолетних культур, для всех нанов можно указать значение = 0, так мы сможем отделять многолетние культуры от однолетних

In [ ]:
df_fi['crop_yearold'] = df_fi['crop_yearold'].fillna(0)
df_fi['crop_yearold'].isna().sum()

In [ ]:
df_fi['fertility_score'].value_counts()

In [ ]:
# Группируем данные и рассчитываем среднее по 'fertility_score' и округляем до целого
fertility_score_mod = df_fi.groupby(['region'])['fertility_score'].agg(lambda x: np.round(x.mean()))
# Заполняем пропуски в 'fertility_score_mod'
df_fi['fertility_score'] = df_fi['fertility_score'].fillna(
    df_fi.apply(lambda x: fertility_score_mod.get((x['region'])), axis=1)
)
# проверим на наличие nan после заполнения, вдруг ошибка какая была
print(df_fi['fertility_score'].isna().sum())

In [ ]:
df_fi['fertility_score'].value_counts()

In [ ]:
# удалим строки, где неизвестна предшествующая культура

df_fi = df_fi.loc[~(df_fi['crop_predecessor'].isna())]
df_fi.isna().sum()

In [ ]:
df_fi['humus'].describe()

In [ ]:
f, ax = plt.subplots(1, 2, figsize=(12, 6))
sns.boxplot(df_fi['humus'], ax=ax[0])
sns.histplot(df_fi['humus'], ax=ax[1])
plt.yscale('log')

### распределение гумуса не поменялось, значит все пиздато

### Посмотрим NaNы

In [ ]:
df_f.drop(columns='fertility_score').corr(numeric_only=1)

### Пусть будет количественная для нашего вопроса, смлишком мало данных с большими годами (потом попробовать объединить до "менее 5 лет" и "5 и более лет" и найти корреляции)
### Дропнули, тк одни наны в корреляциях

### Посмотрим их взаимосвязи с другими характеристиками для решения по их изменению

In [ ]:
def corr_cat(x, y): # функция для подсчета корреляции для 2х категориальных переменных
    confusion_matrix = pd.crosstab(x, y)
    chi2 = chi2_contingency(confusion_matrix)[0]
    n = confusion_matrix.sum().sum()
    phi2 = chi2 / n
    r, k = confusion_matrix.shape
    phi2corr = max(0, phi2 - ((k-1)*(r-1))/(n-1))
    rcorr = r - ( (r-1)**2 / (n-1) )
    kcorr = k - ( (k-1)**2 / (n-1) )
    return np.sqrt(phi2corr / min((kcorr-1), (rcorr-1)))

'''
def coor_cat_num(numeric, categor): # функция для подсчета корреляции для категор/номинал
    df = df_f[[numeric, categor]].dropna()
    mezhgroup = vnutrigroup = 0
    meann = df[numeric].mean()
    for znach in set(df[categor].dropna()):
        data = pd.Series(df[df[categor] == znach][numeric]).dropna()
        vnutrigroup += sum([(elem - data.mean()) ** 2 for elem in data])
        mezhgroup += data.fillna(0).shape[0] * (pd.Series([data.mean()]).fillna(0)[0] - meann) ** 2
        print(znach)
    return (mezhgroup / (mezhgroup + vnutrigroup)) ** 0.5
    '''
def coor_cat_num(df, numeric, categor):
    mezhgroup = vnutrigroup = 0
    meann = df[numeric].mean()
    unique_categories = pd.Series(df[categor]).dropna().unique()
    for znach in unique_categories:
        data = df[df[categor] == znach][numeric].dropna()  # Фильтрация и удаление NaN для каждой категории
        if data.empty:
            continue  # Пропускаем категории без данных
        # Внутригрупповая дисперсия
        vnutrigroup += ((data - data.mean()) ** 2).sum()
        # Межгрупповая дисперсия
        mezhgroup += len(data) * (data.mean() - meann) ** 2
    return (mezhgroup / (mezhgroup + vnutrigroup)) ** 0.5

### Выведем значимые корреляции номинальных с категориальными

In [ ]:
for cat in ['fertility_score', 'crop_predecessor', "hybrid", "reproduction", "seed_origin", "technology"]:
    print(cat + ":")
    for num in ['crop_yearold', 'area', 'yield_plan', 'p2o5', 'k2o', 'humus', 'ph']:
        if coor_cat_num(df_f[[num, cat]].dropna(), num, cat) > 0.3: # чтобы коркоэф имел хотя бы умеренное влияние
            print(f'{num}: {round(coor_cat_num(df_f[[num, cat]].dropna(), num, cat), 3)}', end='\t')
    for categ in ['region', 'land_type', 'season','crop']:
        if corr_cat(df_f[cat], df_f[categ]) > 0.3:
            print(f'{categ}: {round(corr_cat(df_f[cat], df_f[categ]), 3)}', end='\t')
    print()

### С очень большой вероятностью мы правильно подставим данные в crop_predecessor по land_type

In [ ]:
def fill_with_mode(group):
    mode_val = group.mode()
    return mode_val[0] if not mode_val.empty else np.nan

# Заполнение NaN в колонке crop_predecessor на основе land_type
df_fi['crop_predecessor'] = df_fi.groupby('land_type')['crop_predecessor'].transform(fill_with_mode)

###Создадим функцию для замены NaN

In [ ]:
def repl_nan(col, dt_cols):
    global df_fi
    most_common = df_fi.groupby(dt_cols)[col].agg(lambda x: x.mode().iloc[0] if not x.mode().empty else None).rename("most_common")
    df_fi = df_fi.join(most_common, on=dt_cols)
    df_fi[col] = df_fi[col].fillna(df_fi["most_common"])
    df_fi.drop(columns="most_common", inplace=True)  # Удаляем временный столбец, если нужно

### Мы можем попробовать предугадать категорию hybrid из совокупности crop, region (или soil, их коркоэф почти равен 1, без разницы) и crop_yearold

In [ ]:
repl_nan('hybrid', ['crop', 'crop_yearold'])
repl_nan('hybrid', ['crop', 'region'])
repl_nan('hybrid', ['crop_yearold'])
repl_nan('hybrid', ['crop'])
repl_nan('hybrid', ['region'])
for col in df_fi.columns:
    if df_fi[col].isna().any(axis=0):
        print(f'{col}: {df_fi[col].isna().sum()}')

Так как создали функцию, комментим все ручные работы

In [ ]:
'''
most_common = df_fi.groupby(['crop'])['hybrid'].agg(lambda x: x.mode().iloc[0] if not x.mode().empty else None)
df_fi['hybrid'] = df_fi['hybrid'].fillna(
    df_fi.apply(lambda x: most_common.get((x['crop'])), axis=1)
) # при понижении шага с crop_yearold появлялась ошибка OSError: could not get source code, от которой не удалось избавиться, так что перешел на менее точную версию
for col in df_fi.columns:
    if df_fi[col].isna().any(axis=0):
        print(f'{col}: {df_fi[col].isna().sum()}')
'''

In [ ]:
print(f'hybrid: {df_fi["hybrid"].isna().sum()}')

### Далее seed_origin по crop

In [ ]:
repl_nan('seed_origin', ['crop'])
print(f'seed_origin: {df_fi["seed_origin"].isna().sum()}')

### Тк он больше ни с кем нормально не коррелирует, то дропаем эти NaNы

In [ ]:
df_fi = df_fi.dropna(subset='seed_origin')

### Теперь technology

In [ ]:
corr_cat(df_f['hybrid'], df_f["technology"]) # просто проверю

In [ ]:
repl_nan('technology', ['crop', 'region'])
repl_nan('technology', ['hybrid', 'region'])
repl_nan('technology', ['crop'])

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
print(f'technology: {df_fi["technology"].isna().sum()}')

In [ ]:
for col in df_fi.columns:
    if df_fi[col].isna().any(axis=0):
        print(f'{col}: {df_fi[col].isna().sum()}')

yield_plan:

In [ ]:
repl_nan('yield_plan', ['crop_yearold', 'area'])
repl_nan('yield_plan', ['crop_yearold'])

In [ ]:
df_fi['yield_plan_full'] = df_fi.apply(lambda x: x.area * x.yield_plan, axis=1)
f, ax = plt.subplots(1, 1, figsize=(14, 7))
sns.histplot(df_fi['yield_plan_full'])
print(df_fi['yield_plan_full'].describe())

''' Игнат сказал пока что не удалять выбросы
def fkkk(): ## уберем выбросы
    Q1 = df_fi['yield_plan_full'].quantile(0.25)
    Q3 = df_fi['yield_plan_full'].quantile(0.75)
    IQR = Q3 - Q1

    lower_bound = Q1 - 3 * IQR
    upper_bound = Q3 + 3 * IQR

    df_fi['yield_plan_full'] = df_fi['yield_plan_full'].apply(lambda x: x if (x >= lower_bound) & (x <= upper_bound) else np.nan)
    print(df_fi['yield_plan_full'].isna().sum(), df_fi.shape) # на фоне общего размера НаНов немного, значт дропать можно
    df_fi.dropna(subset='yield_plan_full', inplace=True)


fkkk()
sns.histplot(df_fi['yield_plan_full'], ax=ax[1])
plt.show()
'''
df_fi['yield_plan_full_cat'] = df_fi['yield_plan_full'].apply(lambda x: 0.25 if x < df_fi['yield_plan_full'].quantile(0.25)
                                                          else 0.5 if x < df_fi['yield_plan_full'].quantile(0.5)
                                                          else 0.75 if x < df_fi['yield_plan_full'].quantile(0.75)
                                                          else 1)

In [ ]:
repl_nan('reproduction', ['crop', 'crop_yearold'])
'''
# удаляет только десяток из 300 оставшихся нанов + неточно => легче удалить

most_common = df_fi.groupby(['crop', 'yield_plan'])['reproduction'].agg(lambda x: x.mode().iloc[0] if not x.mode().empty else None)
df_fi['reproduction'] = df_fi['reproduction'].fillna(
    df_fi.apply(lambda x: most_common.get((x['crop'], x['yield_plan'])), axis=1)
)
'''
df_fi.dropna(subset='reproduction', inplace=True)

In [ ]:
for col in df_fi.columns:
    if df_fi[col].isna().any(axis=0):
        print(f'{col}: {df_fi[col].isna().sum()}')

________________________________________________________________________________________________________________________

Просто создам функцию для будущего подсчета выбросов

In [ ]:
def del_outs(col):
    iqr = 1.5 * (df_fi[col].quantile(0.75) - df_fi[col].quantile(0.25))
    meann = df_fi[col].mean()
    df_fi[col] = df_fi[col].apply(lambda x: x if meann - iqr <= x <= meann + iqr
                                  else np.nan, axis=1)

________________________________________________________________

Посмотрим различные графики

In [ ]:
df_fi.corr(numeric_only=True)

______________________________________

Посмотрим полную матрицу корреляций:

In [ ]:
corr_matrix = pd.DataFrame(index=df_fi.drop(columns=['key']).columns, columns=df_fi.drop(columns=['key']).columns)
for i in range(corr_matrix.shape[0]):
    for j in range(corr_matrix.shape[0]):
        fir, sec = df_fi.drop(columns=['key']).columns[i], df_fi.drop(columns=['key']).columns[j]
        print(fir, sec)
        if fir == sec:
            corr_matrix.iat[i, j] = 1
            continue
        if df_fi.dtypes[fir] == object:
            if df_fi.dtypes[sec] == object:
                corr_matrix.iat[i, j] = corr_cat(df_fi[fir], df_fi[sec])
            else:
                corr_matrix.iat[i, j] = coor_cat_num(df_fi, sec, fir)
        else:
            if df_fi.dtypes[sec] == object:
                corr_matrix.iat[i, j] = coor_cat_num(df_fi, fir, sec)
            else: corr_matrix.iat[i, j] = df_fi.corr(numeric_only=1)[fir][sec]


In [ ]:
for col in corr_matrix:
    corr_matrix[col] = corr_matrix[col].apply(float)

In [ ]:
sns.clustermap(corr_matrix.round(1), annot=True, cmap='plasma', vmin=-1)

In [ ]:
corr_matrix.to_csv('corr_matrix.csv', index=0)

In [ ]:
corr_matrix.iat[i, j] = corr_cat(df_fi['field'], df_fi['key'])

In [ ]:
sns.pairplot(df_fi)
plt.show()

In [ ]:
dfw.head(2)

In [ ]:
dfw.isna().sum()

In [ ]:
dfw.dtypes

In [ ]:
#dfw['start_date'] = dfw['start_date'].apply(lambda x:datet.strptime(str(x), '%d.%m.%Y %H:%M:%d'))
#dfw['end_date'] = dfw['end_date'].apply(lambda x:datet.strptime(str(x), '%d.%m.%Y'))